In [25]:
import pickle
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm 

In [2]:
with open("news_dict.pickle","rb") as f :
    data = pickle.load(f)

In [13]:
len_list = []
for key, val in data.items():
    len_list.append((key, len(val)))

In [16]:
data2 = sorted(len_list, key = lambda x: x[1],reverse=True)

In [21]:
data2[:10]

[('대상', 65640),
 ('레이', 23658),
 ('SK', 7550),
 ('동양', 7110),
 ('LG', 6228),
 ('신한', 6142),
 ('디오', 6037),
 ('성원', 5473),
 ('동아', 5022),
 ('남성', 4812)]

In [24]:
from konlpy.tag import Mecab
tokenizer = Mecab()

In [27]:
tokenizer.nouns("안녕하세요")

['안녕']

In [28]:
nouns = {}
for key, vals in tqdm(data.items()):
    tot = []
    for txt in vals:
        txt_list = tokenizer.nouns(txt)
        if key in txt_list :
            tot.append(txt_list)
    nouns[key] = tot
    

100%|██████████| 2592/2592 [12:24<00:00,  3.48it/s] 


In [29]:
len_list = []
for key, val in nouns.items():
    len_list.append((key, len(val)))

In [30]:
data2 = sorted(len_list, key = lambda x: x[1],reverse=True)

In [31]:
data2[:10]

[('대상', 61032),
 ('남성', 4512),
 ('삼성전자', 4401),
 ('선진', 2754),
 ('전방', 2546),
 ('한화', 2125),
 ('이마트', 2100),
 ('신한', 2060),
 ('카카오', 1780),
 ('동서', 1683)]

In [32]:
pre_dict = {}
for key, vals in data.items():
    pre_list = []
    if len(vals) == 0:
        continue
    for val in vals:
        val.replace(key,"[MASK]")
        pre_list.append(val)
    pre_dict[key] = pre_list

In [35]:
with open("news_dict_nouns.pickle","wb") as f:
    pickle.dump(nouns, f)